# Parsing Journal of Cognitive Science article data Arthur already scraped (see GDrive)
Aim 1: Parse author & affiliation information for each article.

###### Tentative plans

LOOP THROUGH ALL ROWS: 
- Separate the author and affiliation data
- Create new row for each author in that paper, and denote whether that author was a corresponding author. 
- Also, keep the article name and index (row number) of the article in the original dataset.

In [1]:
import pandas as pd
import string as st

import matplotlib
import numpy as np
from datascience import *
import numpy as np

In [2]:
# Load data CSV -> datascience Table

data_file = '../data/171022_cogsci_journal_unprocessed.csv'
df_pd = pd.read_csv(data_file)

df = Table.read_table(data_file)
index_paperName = Table().with_columns(["Paper Index", df.column(0), "Article Name", df.column(1), "Old Type", df.column(9)])


In [3]:
#Checking article types
#pd.set_option('display.max_rows', 100)
#df_pd.groupby('article_type',sort=True).article_type.count().to_frame().sort_values('article_type',ascending=False)


,article_type
article_type,
Original Article,593
Regular Articles,406
Brief Reports,185
Extended Articles,55
Issue Information,38
Extended Article,37
Letters to the Editor,24
Cognitive Science News,19
Original Articles,19


The 'author_and_details' column in the dataframe contains author and affiliation data.

In [4]:
df_ad = df['authors_and_details'][0:]
df_ad = df_ad
#print(df_ad)

In [ ]:
# Test cell
#df_ad[2].split('\n')#df['authors_and_details'][2]
#print(range(len(df_ad)))

In [5]:
# create list of lists where the number in each sublist is the article index and the length of the 
# sublist is the number of authors for that article
# - i+1 to counter-act data indexing
df_namesplit = []
df_numauth = []
for i in range(1, len(df_ad)):
        
    temp = df_ad[i].split('/n')
    df_namesplit.append(temp)
    df_numauth.append([i+1]*len(temp))

#print(df_numauth)

In [6]:
# Makes list of lists into single list
finList = []
for i in range(len(df_numauth)):
    for j in range(len(df_numauth[i])):
        finList.append(df_numauth[i][j])
        
#print(finList)

In [7]:
# This function takes a string (author of article ) and strips "Corresponding Author" -> returns stripped string
def clearCorresponding(author):
    result = []
    if ',Corresponding author' in author:

        endIndex = author.find(',')
        result.append(author[:endIndex])
        return result
    
    elif 'Corresponding author' in author:
        endIndex = author.find('Corresponding author')
        
        result.append(author[:endIndex])
        return result
   
    else:
        return author

In [8]:
# Main algorithm to construct table of Articles and Authors

results = Table(make_array("Article Id", "Author", "Department/University", "Corresponding Author"))
# For each sublist in the list of lists df_namesplit
articleCounter = 0 # Used to iterate through list of article id's
for i in range(len(df_namesplit)):
    
    articlesAuthors = df_namesplit[i]
    
    for j in range(len(articlesAuthors)):
        corr_auth = 0
        element = df_namesplit[i][j].split('             ')
        if len(element) > 1: # If the article has an author and the corresponding department/ university
    
            author = clearCorresponding(element[0])
            dept = element[1]
            
            if("Corresponding author" in df_namesplit[i][j]):
                corr_auth = 1
            results = results.with_row([finList[articleCounter],author, dept, corr_auth])
            articleCounter += 1
            
        else:            
            author = clearCorresponding(element[0])
            dept = "NaN"
            
            #newRow = make_array(corr_auth,author, dept)
            results = results.with_row([finList[articleCounter],author, dept, corr_auth])
            articleCounter += 1
            

In [9]:
# Join article name with article id number
results = results.join("Article Id", index_paperName, "Paper Index")
# not sure why we need to drop column 5 twice
results.show()

Article Id,Author,Department/University,Corresponding Author,Article Name,Old Type
2,Allan Collins,NaN,0,Why Cognitive Science,Original Article
3,Daniel G. Bobrow,Xerox Palo Alto Research Center,1,"An Overview of KRL, a Knowledge Representation Language",Original Article
3,Terry Winograd,Stanford University,0,"An Overview of KRL, a Knowledge Representation Language",Original Article
4,Wendy Lehnert,Yale University,1,Human and Computational Question Answering†,Original Article
5,Andrew Ortony,University of Illinois at Urbana-Champaign,1,Definite Descriptions and Semantic Memory†,Original Article
5,Richard C. Anderson,University of Illinois at Urbana-Champaign,0,Definite Descriptions and Semantic Memory†,Original Article
6,Professor Ira Goldstein,Massachusetts Institute of Technology,1,"Artificial Intelligence, Language, and the Study of Know ...",Original Article
6,Seymour Papert,Massachusetts Institute of Technology,0,"Artificial Intelligence, Language, and the Study of Know ...",Original Article
7,John R. Anderson,Yale University,1,Induction of Augmented Transition Networks†,Original Article
8,Jerome A. Feldman,University of Rochester,1,Decision Theory and Artificial Intelligence II: The Hung ...,Original Article


In [10]:
Extended_Article = ["Extended Articles", "Extended Article", "Extended article", "Extended articles"]
Original_Article = ["Original Article", "Original Articles"]
Regular_Article = ["Regular Articles", "Regular articles", "Regular Article", "Regular article"]
Letter_to_Editor = ["Letters to the Editor","Letter to the Editor"]
Brief_Report = ["Brief Reports", "Brief Report", "Brief reports", "Brief report"]
Announcement = ["Announcement"]
Special = ["2011 Rumelhart Prize Special Issue Honoring Judea Pearl","SPECIAL ISSUE: 2009 Rumelhart Prize Special Issue Honoring Susan Carey","2010 Rumelhart Prize Special Issue Honoring James L. McClelland","SPECIAL ISSUE: 2007 Rumelhart Prize Special Issue Honoring Jeffrey L. Elman Language as a Dynamical System"]
Comment = ["Comment","Short Communication"]
Commentary = ["Commentary"]


In [11]:
def consolidate_Type (oldType):
    if oldType in Extended_Article:
        return "Extended Article"
    if oldType in Original_Article:
        return "Original Article"
    if oldType in Regular_Article:
        return "Regular Article"
    if oldType in Brief_Report:
        return "Brief Report"
    if oldType in Letter_to_Editor:
        return "Letter to Editor"
    if oldType in Announcement:
        return "Announcement"
    if oldType in Special:
        return "Special"
    if oldType in Comment:
        return "Comment"
    if oldType in Commentary:
        return "Commentary"
    return "null"

In [12]:
results = results.with_column("New Type", results.apply(consolidate_Type, "Old Type"))
results = results.drop("Old Type")


In [13]:
#results.show(1000)

In [14]:
results2 = Table().with_columns(["Article Id",[], "Author", [], "Department/University", [], "Corresponding Author",[], "Article Name", [], "Type", []])
#results2

In [15]:

for i in range(results.num_rows):
    if results.column(5)[i] != "null":
        results2 = results2.with_row(results.row(i))  
#results2.show()

Article Id,Author,Department/University,Corresponding Author,Article Name,Type
2,Allan Collins,NaN,0,Why Cognitive Science,Original Article
3,Daniel G. Bobrow,Xerox Palo Alto Research Center,1,"An Overview of KRL, a Knowledge Representation Language",Original Article
3,Terry Winograd,Stanford University,0,"An Overview of KRL, a Knowledge Representation Language",Original Article
4,Wendy Lehnert,Yale University,1,Human and Computational Question Answering†,Original Article
5,Andrew Ortony,University of Illinois at Urbana-Champaign,1,Definite Descriptions and Semantic Memory†,Original Article
5,Richard C. Anderson,University of Illinois at Urbana-Champaign,0,Definite Descriptions and Semantic Memory†,Original Article
6,Professor Ira Goldstein,Massachusetts Institute of Technology,1,"Artificial Intelligence, Language, and the Study of Know ...",Original Article
6,Seymour Papert,Massachusetts Institute of Technology,0,"Artificial Intelligence, Language, and the Study of Know ...",Original Article
7,John R. Anderson,Yale University,1,Induction of Augmented Transition Networks†,Original Article
8,Jerome A. Feldman,University of Rochester,1,Decision Theory and Artificial Intelligence II: The Hung ...,Original Article


In [16]:
results2.to_csv("cleaned_data")

In [17]:
#Sorting by hexagonal terms code 

In [ ]:
results2 = results2.with_column("Affiliation",[])

In [ ]:
# issue Computational Neuroscience
compSci = ["artificial intelligence", "computer science", "informatics", "computer", "cybernetics", "computing", "eecs", "technology"]

In [ ]:
def consolodateAffiliation(aff):
    temp = aff.lower()
    
    if "cognitive" in temp or "cognition" in temp:
        return "Cognitive Science"
    if "psychology" in temp or "psychological" in temp:
        return "Psychology"
    if "philosophy" in temp:
        return "Philosophy"
    if "linguistics" in temp:
        return "Linguistics"
    if "neuro" in temp or "brain" in temp:
        return "Neuroscience"
    for i in compSci:
        if i in temp:
            return "Artificial Intelligence"
    if "anthropology" in temp:
        return "Anthropology"
    if "social science" in temp:
        return "Social Science"
    return "Other"

In [ ]:
results2 = results2.with_column("aff",results2.apply(consolodateAffiliation, 2))

In [ ]:
# HOW TO COUNT AFFILIATIONS
print(results2.where("aff", are.equal_to("Cognitive Science")).num_rows)
print(results2.where("aff", are.equal_to("Psychology")).num_rows)
print(results2.where("aff", are.equal_to("Philosophy")).num_rows)
print(results2.where("aff", are.equal_to("Artificial Intelligence")).num_rows)
print(results2.where("aff", are.equal_to("Anthropology")).num_rows)
print(results2.where("aff", are.equal_to("Neuroscience")).num_rows)
print(results2.where("aff", are.equal_to("Linguistics")).num_rows)
print(results2.where("aff", are.equal_to("Social Science")).num_rows)
print(results2.where("aff", are.equal_to("Other")).num_rows)

In [ ]:
#Logic, Language and Computation,
#physics
#decision technologies
#neural computation
#psycholinguistics (french?)

#classify departments into multiple bins
others = results2.where("aff", are.equal_to("Other"))
others.select(2).show()

In [ ]:
# All below cells are tests to see what the data is doing

In [ ]:
df_dept = pd.read_csv("cleaned_data")
colnames= ["Article Id","Author","Department_University","Corresponding Author","Article Name","Type"]
df_dept_new=pd.read_csv("cleaned_data", names=colnames)

In [ ]:
bad_inds = np.where(df_dept_new['Department_University'].isnull())[0]
len(bad_inds)
#len(df_dept_new)

In [ ]:
df_pd.iloc[368]['authors_and_details']

In [ ]:
df_dept_new.iloc[bad_inds]

In [ ]:
pd.set_option('display.max_rows', 100)
df_dept_new.groupby('Department_University',sort=True).Department_University.count().to_frame().sort_values('Department_University',ascending=False)

In [ ]:
# index the row (e.g. 2nd) and split by /n character
df['authors_and_details'][2].split('/n')

In [ ]:
# the 0th element contains the information of the first author
df['authors_and_details'][2].split('/n')[0]

In [ ]:
# further split author info to separate name and affiliation
au1 = df['authors_and_details'][2].split('/n')[0].split('             ')

In [ ]:
au1

In [ ]:
# find the corresponding author and delete that part of the string ',Corresponding author'
if 'Corresponding author' in au1[0]:
    endind = au1[0].find(',Corresponding author')
    print(endind)
    print(au1[0][:endind], '- ', au1[1])

In [ ]:
# make a new dataset


### The latter ones have a little more information
Let's go to index 1500

In [ ]:
df['authors_and_details'][1501].split('/n')

In [ ]:
df['article_name'][1501]